## 1. Parteiprogramme

Die Aufgabe ist die Replikation und Erweiterung dieser Analyse für die BTW 2021. Der unten stehende Code kann für das Umwandeln einer beliebigen PDF für einen gegebenen URL genutzt werden.

Nutzen Sie in ihren Analysen geeignete Stopwörter und erweitern Sie diese wo es Ihnen sinnvoll erscheint. 

Die 20 häufigsten Begriffe je Partei, identifizieren der wichtigsten Begriffe (für die Wahl gesamt und für die jeweiligen Parteien separat). Nutzen Sie wo möglich Visualisierungen um Ihre Ergebnisse zu verdeutlichen.

In [1]:
library(tidyverse)

NameError: name 'library' is not defined

In [2]:
pdf_path <- "https://www.spd.de/fileadmin/Dokumente/Beschluesse/Programm/SPD-Zukunftsprogramm.pdf"
# extract text
get_txt_output =function(url){
  pdftools::pdf_text(url) %>%
  paste(sep = " ") %>%
  stringr::str_replace_all(fixed("\n"), " ") %>%
  stringr::str_replace_all(fixed("\r"), " ") %>%
  stringr::str_replace_all(fixed("\t"), " ") %>%
  stringr::str_replace_all(fixed("\""), " ") %>%
  paste(sep = " ", collapse = " ") %>%
  stringr::str_squish() %>%
  stringr::str_replace_all("- ", "")}

In [3]:
spd = get_txt_output("https://www.spd.de/fileadmin/Dokumente/Beschluesse/Programm/SPD-Zukunftsprogramm.pdf")
#glimpse(spd)
cdu = get_txt_output("https://www.csu.de/common/download/Regierungsprogramm.pdf")
#glimpse(cdu)
gruene = get_txt_output("https://cms.gruene.de/uploads/documents/Wahlprogramm-DIE-GRUENEN-Bundestagswahl-2021_barrierefrei.pdf")
fdp = get_txt_output("https://www.fdp.de/sites/default/files/2021-08/FDP_BTW2021_Wahlprogramm_1.pdf")
afd = get_txt_output("https://www.afd.de/wp-content/uploads/sites/111/2021/06/20210611_AfD_Programm_2021.pdf")
linke = get_txt_output("https://www.die-linke.de/fileadmin/download/wahlen2021/Wahlprogramm/DIE_LINKE_Wahlprogramm_zur_Bundestagswahl_2021.pdf")
csu = get_txt_output("https://www.csu.de/common/download/CSU-Programm_Gut_fuer_Bayern_Gut_fuer_Deutschland_final.pdf")
fw = get_txt_output("https://www.freiewaehler.eu/template/elemente/203/FREIE%20WÄHLER_Wahlprogramm-BTW21.pdf")

In [4]:
library(tidytext)

In [5]:
parties <- c("spd", "cdu", "gruene", "fdp", "afd", "linke", "csu", "fw")
programm <- list(spd, cdu, gruene, fdp, afd, linke, csu, fw)
programm %>%
    set_names(parties) %>%
    map_df(as_tibble, .id = "party") %>%
    mutate(party = factor(party, levels = parties)) %>%
    drop_na(value) %>%
    group_by(party) %>%
    unnest_tokens(word, value) -> df_programm
tail(df_programm)

In [32]:
library("SnowballC")
library("NLP")
# Liste von Stop-Words erstellen
library("lsa")
library("tm")

data(stopwords_de)
stopwordsLsa <- stopwords_de
stopwordsTm <- stopwords("german")

ownStopwords <- c("abs", "afd", "ar", "ange", "aus-", "at", "ät", "ärk", "am", "äm",
                  "bundestagswahl", "bundestagswahlprogramm", "bündnis", "be", "bun", "bne", "bek", "bet",
                  "cdu", "csu", "ckeln", "chen",
                  "dabei", "dafür", "demokraten", "demo-", "deut",
                  "el", "ent", "ei", "ezb", "en", "elt", "ert", "erk", "efs", "eit",
                  "fdp", "freie", "ff", "fe", "fam",
                  "gg", "grün", "grüne", "grünen", "ge",
                  "innen", "insbesondere", "in", "inno", "In-", "ik", "irt", "ion", "int", "ick", "ie",
                  "kapitel", "künftig",
                  "liberalen", "linke", "len", "le",
                  "men",
                  "ner", "nen",
                  "öffent-", "om", "priv", "on",
                  "regierungsprogramm", "ruk", "rbeit",
                  "seither", "siehe", "sowie", "sozialdemokraten", "sozialdemokratinnen",
                  "sozialdemokratische", "sozialdemokratischer", "si", "st", "sv", "schüt",
                  "spd",
                  "ten",
                  "unterneh", "ung", "u.a", "unt", "ur", "ult", "üt",
                  "vgl", "vi", "vii", "viii", "vn", "ver", "ver-",
                  "wahlprogramm", "wer", "wol", "wei", "wirt",
                  "z.b", "zudem", "zukunftsprogramm")

stop_words <- tibble(word = unique(c(stopwordsLsa, stopwordsTm, ownStopwords)))
head(stop_words)

In [23]:
library("forcats")
library("stringr")
df_programm %>%
    anti_join(stop_words, by = "word") %>%
    mutate(word = str_replace_all(word, "[[:digit:]]", "")) %>%
    filter(word != "", word != ".", word != "..") %>%
    filter(nchar(word, type = "chars") > 1) -> clear_programm
head(clear_programm)

In [24]:
clear_programm %>% 
  count(word, sort = TRUE) %>% 
  head(n = 20) %>% 
  mutate(word = reorder(word, n)) %>% 
  ggplot(aes(word, n, fill = factor(word))) +
  geom_col() +
  labs(x = NULL,y = "Anzahl", title = "Häufigste Begriffe (alle Parteien)") +
  coord_flip() + theme(legend.position = "none")

In [40]:
install.packages("gsubfn")
install.packages("proto")
library(gsubfn)
library(proto)
library(ggplot2)

In [ ]:
txt %>%
    filter(party == p) %>% 
    count(words, sort = TRUE) %>% 
    head(n = 20) %>% 
    mutate(words = reorder(words, n)) %>% 
    ggplot(aes(words, n)) +
    geom_col(fill = partyColor[p], color = partyColor[p]) + 
    labs(y = "Anzahl", x = NULL, title = p) +
    theme(axis.text = element_text(size = 10)) +
    coord_flip()

In [58]:
clear_programm %>%
  group_by(party) %>%
  count(word, sort=TRUE) %>%
  arrange(desc(n)) %>%
  top_n(20, n) %>%
  ungroup() %>%
  mutate(word = reorder(word, n)) %>%
  ggplot(aes(x=word, y=n, fill = factor(party))) +
  geom_col() + 
  coord_flip() +
  theme_minimal() +
  facet_wrap(~party, scales="free") +
  labs(x = NULL, y = "Anzahl", title = "Häufigste Begriffe (je Partei)") + 
  theme(legend.position = "none") -> plot
plot

In [53]:
clear_programm %>%
        anti_join(stop_words, by = "word") %>%
        count(party, word, sort = TRUE) %>%
        ungroup() -> words

words %>%
        bind_tf_idf(word, party, n) -> words_tf
 
words_tf %>%
  group_by(party) %>%
  arrange(desc(tf_idf)) %>%
  top_n(20, tf_idf) %>%
  ungroup() %>%
  mutate(word = reorder(word, tf_idf)) %>%
  ggplot(aes(x=word, y=tf_idf, fill = factor(party))) +
  geom_col() + 
  coord_flip() + theme_minimal() +
  facet_wrap(~party, scales="free") +
  labs(x = NULL, y = "tf-idf", title = "Wichtigste Begriffe (je Partei)") + 
  theme(axis.text.x = element_text(angle = 45, hjust = 1, size = 10)) +
  theme(legend.position = "none") -> plot
plot

## Aufgabe 2: Hogwarts und Co

Das `harrypotter` package bietet einen komfortablen Zugriff auf den Volltext aus allen Harrypotter Büchern:

> Each text is in a *character vector* with each element representing a *single chapter*. The package contains:

* `philosophers_stone`: Harry Potter and the Philosophers Stone, published in 1997
* `chamber_of_secrets`: Harry Potter and the Chamber of Secrets, published in 1998
* `prisoner_of_azkaban`: Harry Potter and the Prisoner of Azkaban, published in 1999
* `goblet_of_fire`: Harry Potter and the Goblet of Fire, published in 2000
* `order_of_the_phoenix`: Harry Potter and the Order of the Phoenix, published in 2003
* `half_blood_prince`: Harry Potter and the Half-Blood Prince, published in 2005
* `deathly_hallows`: Harry Potter and the Deathly Hallows, published in 2007

Nutzen Sie die Daten um folgendes umzusetzen:

* Eine Visualisierung des Plot Sentiments über die Bücher. Stellen Sie diese bitte in einem Plot dar und sorgen Sie dafür das Anfang und Ende der Bücher auf der gleichen x-Koordinate liegen. Annotieren Sie besonders negative und positive Kapitel mit Band und Kapitelnummer und Kapitelname.
* Eine kapitelbasierte Klassifikation zu welchem Buch ein Kapitel gehört.

In [181]:
install.packages("devtools")

In [182]:
devtools::install_github("bradleyboehmke/harrypotter")

In [183]:
library(harrypotter)

In [184]:
hp_books <- c("philosophers_stone", "chamber_of_secrets", "prisoner_of_azkaban", 
              "goblet_of_fire", "order_of_the_phoenix", "half_blood_prince", 
              "deathly_hallows")
hp_words <- list(philosophers_stone, chamber_of_secrets, prisoner_of_azkaban,
              goblet_of_fire, order_of_the_phoenix, half_blood_prince, deathly_hallows)
hp_words %>%
  set_names(hp_books) %>%
  map_df(as_tibble, .id = "book") %>%
  mutate(book = factor(book, levels = hp_books)) %>%
  drop_na(value) %>%
  group_by(book) %>%
  mutate(chapter = row_number(book)) %>%
  ungroup() %>%
  unnest_tokens(word, value) -> df_hp
head(df_hp)

In [185]:
afinn <- read_csv("https://raw.githubusercontent.com/jboscomendoza/rpubs/master/sentimientos_afinn/lexico_afinn.en.es.csv")
names(afinn)[names(afinn) == 'Word'] <- 'word'
head(afinn)

In [186]:
df_hp %>%
  inner_join(afinn, by="word") %>%
  group_by(book, chapter) %>%
  summarize(value = sum(Puntuacion)) %>%
  ggplot(mapping = aes(x = chapter, y = value, fill = book)) +
  geom_col() +
  facet_wrap(facets = vars(book), scales = "free_x") +
  labs(title = "Visualisierung des Plot Sentiments über die Bücher",
        x = "Chapter",
        y = "Emotional score") + 
  theme(legend.position = "none")

In [187]:
df_hp %>%
  inner_join(afinn, by="word") %>%
  group_by(book) %>%
  mutate(cumvalue = cumsum(Puntuacion)) %>%
  ggplot(mapping = aes(x = chapter, y = cumvalue, fill = book)) +
  geom_step() +
  facet_wrap(facets = vars(book), scales = "free_x") +
  labs(title = "Visualisierung des Plot Sentiments über die Bücher",
    x = "Chapter",
    y = "Cumulative emotional value")

In [190]:
df_hp %>%
  filter(chapter>=1) %>%
  inner_join(afinn, by="word") %>%
  group_by(book, chapter) %>%
  summarize(Sentiment=mean(Puntuacion)) %>%
  ggplot(aes(x = chapter, y = Sentiment), size=2) +
  geom_step() + facet_wrap(facets = vars(book), scales = "free_x", ncol=4) +
  geom_hline(aes(yintercept=0)) + theme_minimal() +
  theme(legend.position="none")

## Aufgabe 3: State of the Union

Unter https://www.presidency.ucsb.edu/documents/presidential-documents-archive-guidebook/annual-messages-congress-the-state-the-union#Table%20of%20SOTU können Sie komfortabel auf alle State of the Union Ansprachen der US-Präsidenten zugreifen.

Sammeln Sie die Daten mit `rvest` und setzen Sie mit den Daten einen *tf-idf* Vergleich zwischen dem 19. und 20. Jahrhundert um.

Achtung: die mehrfachen Ansprachen von Nixon im Jahr 1973 können Sie ignorieren - für die Bereinigung der URL-Liste kann dieser Filter hilfreich sein: `filter(grepl("pid",url))``

In [143]:
speech = read_csv('../input/us-state-of-the-union-addresses-1790-2019/state_ofthe_union_texts.csv')
spec(speech)

In [144]:
speech %>%
    select(-Title) %>%
    filter(Year>1800 & Year<1901) -> speech19
#head(speech19)

In [213]:
speech19 %>%
    unnest_tokens(word, Text) -> tidy_speech19
#head(tidy_speech19)

In [246]:
data("stop_words")
ownStopwords <- c("nand", "nthe", "nthat", "nfor", "nwhich", "nhave")
stop_words1 <- tibble(word = unique(c(stop_words, ownStopwords)))

text_df <- tibble(line = 1:6, word = ownStopwords)
#text_df

In [243]:
tidy_speech19 %>%
    anti_join(text_df, by="word") %>%
    anti_join(stop_words, by="word") -> tidy_speech19_words
head(tidy_speech19_words)

In [230]:
tidy_speech19_words %>% 
  count(word, sort = TRUE) %>% 
  head(n = 10) %>% 
  mutate(word = reorder(word, n)) %>% 
  ggplot(aes(word, n, fill = factor(word))) +
  geom_col() +
  labs(x = NULL,y = "Anzahl", title = "Häufigste Begriffe in presidential speeches (19th century)") +
  coord_flip() + theme(legend.position = "none")

In [161]:
install.packages("ggwordcloud")


In [163]:
library(ggwordcloud)

In [174]:
set.seed(123) # ensure reproducibility of the wordcloud
tidy_speech19_words %>%
  count(word) %>%
  # keep only top 100 words for wordcloud
  slice_max(order_by = n, n = 100) %>%
  mutate(angle = 90 * sample(c(0, 1), n(), replace = TRUE, prob = c(70, 30))) %>%
  ggplot(aes(label = word, size = n, angle = angle, fill = word)) +
  geom_text_wordcloud(rm_outside = TRUE) +
  scale_size_area(max_size = 15) +
  ggtitle("Most frequent tokens in presidential speeches (19th century)") + theme_minimal()

In [244]:
# Häufigkeit der Wörter 
wrdCount <- tidy_speech19_words %>% 
  count(President, word, sort = TRUE) %>% 
  ungroup

# Wortanzahl 
ttlWords <- wrdCount %>% 
  group_by(President) %>% 
  summarise(total = sum(n))

wrdCount <- left_join(wrdCount, ttlWords, by = "President")

# Berechnung des tf-idf-Maßes
wrdCount <- wrdCount %>% 
  mutate(word = str_replace_all(word, "[[:digit:]]", "")) %>%
  filter(word != "", word != ".", word != "..") %>%
  filter(nchar(word, type = "chars") > 3) %>%
  bind_tf_idf(word, President, n)

In [245]:
plotwrds <- wrdCount %>%
  arrange(desc(tf_idf)) %>%
  mutate(word = factor(word, levels = rev(unique(word))))

plotwrds %>% 
  top_n(15) %>%
  ggplot(aes(word, tf_idf, fill = President)) +
  geom_col() +
  labs(x = NULL, y = "tf-idf", fill = "President",
       title = "Wichtigste Begriffe (alle Präsidenten) 19th century") +
  coord_flip()

In [231]:
speech %>%
    select(-Title) %>%
    filter(Year>1900 & Year<2001) -> speech20
#head(speech20)

In [232]:
speech20 %>%
    unnest_tokens(word, Text) -> tidy_speech20
head(tidy_speech20)

In [233]:
tidy_speech20 %>%
    anti_join(text_df, by="word") %>%
    anti_join(stop_words, by="word") -> tidy_speech20_words
head(tidy_speech20_words)

In [234]:
tidy_speech20_words %>% 
  count(word, sort = TRUE) %>% 
  head(n = 10) %>% 
  mutate(word = reorder(word, n)) %>% 
  ggplot(aes(word, n, fill = factor(word))) +
  geom_col() +
  labs(x = NULL,y = "Anzahl", title = "Häufigste Begriffe in presidential speeches (20th century)") +
  coord_flip() + theme(legend.position = "none")

In [180]:
set.seed(123) # ensure reproducibility of the wordcloud
tidy_speech20_words %>%
  count(word) %>%
  # keep only top 100 words for wordcloud
  slice_max(order_by = n, n = 100) %>%
  mutate(angle = 90 * sample(c(0, 1), n(), replace = TRUE, prob = c(70, 30))) %>%
  ggplot(aes(label = word, size = n, angle = angle, fill = word)) +
  geom_text_wordcloud(rm_outside = TRUE) +
  scale_size_area(max_size = 15) +
  ggtitle("Most frequent tokens in presidential speeches (20th century)") + theme_minimal()

In [239]:
# Häufigkeit der Wörter 
wrdCount <- tidy_speech20_words %>% 
  count(President, word, sort = TRUE) %>% 
  ungroup

# Wortanzahl 
ttlWords <- wrdCount %>% 
  group_by(President) %>% 
  summarise(total = sum(n))

wrdCount <- left_join(wrdCount, ttlWords, by = "President")

# Berechnung des tf-idf-Maßes
wrdCount <- wrdCount %>% 
  mutate(word = str_replace_all(word, "[[:digit:]]", "")) %>%
  filter(word != "", word != ".", word != "..") %>%
  filter(nchar(word, type = "chars") > 3) %>%
  bind_tf_idf(word, President, n)

In [241]:
plotwrds <- wrdCount %>%
  arrange(desc(tf_idf)) %>%
  mutate(word = factor(word, levels = rev(unique(word))))

plotwrds %>% 
  top_n(15) %>%
  ggplot(aes(word, tf_idf, fill = President)) +
  geom_col() +
  labs(x = NULL, y = "tf-idf", fill = "President",
       title = "Wichtigste Begriffe (alle Präsidenten) 20th century") +
  coord_flip()